In [367]:
from ipywidgets import widgets
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import json
import numpy as np
from copy import deepcopy
import os

In [396]:
# general params

model_name = widgets.Dropdown(
    description='model name',
    value='BoTorch',
    options=['GPyOpt', 'Gadma', 'BoTorch'])

objective_name = widgets.Dropdown(
    description='Objective name', 
    value='1_Bot_4_Sim',
    options=['1_Bot_4_Sim', '2_ExpDivNoMig_5_Sim', '2_DivMig_5_Sim'])

num_run = widgets.BoundedIntText(
    value=50,
    min=1,
    max=100,
    step=1,
    description='num run')

num_evaluations = widgets.BoundedIntText(
    value=300,
    min=10,
    max=400,
    step=10,
    description='num evaluations')

model_num = widgets.BoundedIntText(
    value=1,
    min=1,
    max=100,
    step=1,
    description='model_num')

button_add = widgets.ToggleButton(
    description='Add',
    button_style='success')

button_reset = widgets.ToggleButton(
    description='Reset',
    button_style='warning')

button_clear = widgets.ToggleButton(
    description='Clear',
    button_style='danger')

# table
model_kwargs_table = go.FigureWidget(data=[go.Table(
    header=dict(values=['parameter', 'value', 'type']), 
    cells=dict(values=[]))],
    layout={'title': 'model params', 
            'width': 500})

model_param = widgets.Text(
    description='Param')

model_value = widgets.Text(
    description='Value')

model_param_type = widgets.Dropdown(
    value='string',
    description='Type',
    options=['int', 'float', 'string'])

button_add_model_params = widgets.ToggleButton(
    description='Add',
    button_style='success')

button_reset_model_params = widgets.ToggleButton(
    description='Reset',
    button_style='warning'
)

button_reset_table = widgets.ToggleButton(
    description='Reset',
    button_style='warning')


# construct container

container1 = widgets.VBox([model_name, objective_name])
container2 = widgets.VBox([num_run, num_evaluations, model_num])
container3 = widgets.HBox([container1, container2])
buttons = widgets.HBox([button_add, button_reset, button_clear])

lower_container = widgets.VBox([container3, buttons])

buttons_params = widgets.HBox([button_add_model_params, button_reset_model_params])
container_param = widgets.VBox([model_param, model_value, model_param_type, buttons_params])
upper_container = widgets.VBox([widgets.HBox([model_kwargs_table, container_param]),
                               button_reset_table])

container = widgets.VBox([upper_container, lower_container])

In [397]:
def response_add_model_params(change):
    table = list(deepcopy(model_kwargs_table.data[0].cells.values))
    if (table == []):
        table = [[], [], []]
    table[0].append(model_param.value)
    table[1].append(model_value.value)
    table[2].append(model_param_type.value)
    model_kwargs_table.data[0].cells.values = table
    
def response_reset_model_params(change):
    model_param.value = ''
    model_value.value = ''
    model_param_type.value = 'string'

def response_reset_table(change):
    model_kwargs_table.data[0].cells.values = []
    response_reset_model_params(change)
    


def response_add(change):
    with open("models_to_run.json", 'r') as file:
        models_to_run = json.load(file)
    
    
    model_params = dict()
    if (not len(model_kwargs_table.data[0].cells.values) == 0):
        for value, param, _type in zip(model_kwargs_table.data[0].cells.values[0], 
                                       model_kwargs_table.data[0].cells.values[1],
                                       model_kwargs_table.data[0].cells.values[2]):

            if (_type == 'int'):
                param = int(param)
            elif (_type == 'float'):
                param = float(param)
            model_params[value] = param
    
    models_to_run.append([model_name.value, objective_name.value, num_run.value, num_evaluations.value,
                         model_num.value, model_params])
    with open("models_to_run.json", 'w') as file:
        json.dump(models_to_run, file)
        

def response_reset(change):
    model_name.value = 'BoTorch'
    objective_name.value = '1_Bot_4_Sim'
    num_run.value=50
    num_evaluations.value=300
    model_num.value=1

def response_clear(change):
    with open("models_to_run.json", 'w') as file:
        json.dump([], file)
        

# suggest model num

def suggest_model_num(change):
    path = f"../data/log/{objective_name.value}/{model_name.value}/{num_evaluations.value}_evaluations"
    if not os.path.exists(path):
        model_num.value = 1
    else:
        max_num = 0
        for name in os.listdir(path):
            assert name[:5] == 'model'
            max_num = max(max_num, int(name[5:]))
        model_num.value = max_num + 1

In [398]:
button_add_model_params.observe(response_add_model_params, names='value')
button_reset_model_params.observe(response_reset_model_params, names='value')

button_reset_table.observe(response_reset_table, names='value')

button_add.observe(response_add, names='value')
button_reset.observe(response_reset, names='value')
button_clear.observe(response_clear, names='value')

# suggest model num
suggest_model_num(None)
model_name.observe(suggest_model_num, names='value')
objective_name.observe(suggest_model_num, names='value')
num_evaluations.observe(suggest_model_num, names='value')

In [399]:
container

    'data': [{'cells': {'values': []},
           …